In [ ]:
from tensorflow.keras.datasets import boston_housing

In [ ]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data[0]

In [ ]:
train_targets[0]

In [ ]:
mean = train_data.mean(axis=0)

In [ ]:
mean

In [ ]:
train_data -= mean

In [ ]:
std = train_data.std(axis=0)

In [ ]:
std

In [ ]:
train_data /= std

In [ ]:
train_data[0]

In [ ]:
test_data -= mean

In [ ]:
test_data /= std

In [ ]:
train_data[0]

In [ ]:
test_data[0]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop',loss='mse',metrics=['mae',])
    return model

In [ ]:
import numpy as np

k = 4
num_val_samples = len(train_data) //4
num_epochs = 500
#all_scores = []
all_mae_histories = []

In [ ]:
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]], axis = 0)

    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]], axis = 0)

    model = build_model()
    # model.fit(partial_train_data, partial_train_targets,
    #           epochs = num_epochs, batch_size = 1, verbose = 0)

    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data = (val_data, val_targets),
                        epochs = num_epochs, batch_size = 1, verbose = 0)

    #val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    #all_scores.append(val_mae)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
len(all_mae_histories)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
#plt.plot(range(1, 101), average_mae_history[:100])
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=10)

In [ ]:
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mse_score

In [ ]:
test_mae_score